# Transform addresses data
1. Create one record for each customer with 2 sets of address columns, 1 for shipping and 1 for billing address
2. Write transformed data to Silver schema

In [0]:
%sql
select 
  customer_id,
  address_type,
  address_line_1,
  city,
  state,
  postcode
from gizmobox.bronze.v_addresses;

## 1. Pivot clause
Documentation: https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/sql-ref-syntax-qry-select-pivot

In [0]:
%sql
select
  *
from (
  select 
    customer_id,
    address_type,
    address_line_1,
    city,
    state,
    postcode
  from gizmobox.bronze.v_addresses
)
pivot (
  max(address_line_1) as address_line_1,
  max(city) as city,
  max(state) as state,
  max(postcode) as postcode
  for address_type in ('shipping', 'billing')
);


## 2. Write transformed data to Silver schema

In [0]:
%sql
create or replace table gizmobox.silver.addresses
as
select
  *
from (
  select 
    customer_id,
    address_type,
    address_line_1,
    city,
    state,
    postcode
  from gizmobox.bronze.v_addresses
)
pivot (
  max(address_line_1) as address_line_1,
  max(city) as city,
  max(state) as state,
  max(postcode) as postcode
  for address_type in ('shipping', 'billing')
);


In [0]:
%sql
select * 
from gizmobox.silver.addresses